# Primer Pipeline de RAG para BERT

**Objetivo:** Evaluar la especialización de BERT en el dominio biomédico mediante una estrategia de RAG.

**Método:**

* Base de Documentos: dataset de abstracts etiquetados de PubMed.
  * Crudo: [abstracts-12-05.csv](bert-ft-data/pubmed-hpo-pa-corpus/abstracts/abstracts-12-05.csv).
  * Index train: [index-31-05-train.csv](bert-ft-data/pubmed-hpo-pa-corpus/phenotypes/index-31-05-train.csv)
  * Index test: [index-31-05-test.csv](bert-ft-data/pubmed-hpo-pa-corpus/phenotypes/index-31-05-test.csv)
  * Clean: [abstracts](bert-ft-data/pubmed-hpo-pa-corpus/abstracts/abstracts-31-05-train.csv)
* Conjuntos de evaluacioń: pares de abstracts etiquetados con su similitud Lin en HPO.
  * train: [train_pairs-31-05.csv](bert-ft-data/evaluation/train_pairs-31-05.csv)
  * test:  [test_pairs-31-05.csv](bert-ft-data/evaluation/test_pairs-31-05.csv)

In [1]:
# 1. Cargar entorno

# Load env

# Cargar .env
import os
from dotenv import load_dotenv
from getpass import getpass
import sys
from IPython.core.getipython import get_ipython

notebook_path = os.path.realpath(os.path.join(os.getcwd(), get_ipython().starting_dir))
WS_ROUTE = (os.path.dirname(notebook_path)) + "/"
load_dotenv(WS_ROUTE + "env/diic.env")

if "HF_API_KEY" not in os.environ:
    os.environ["HF_API_KEY"] = getpass("Enter Hugging Face API key:")

if "EMBEDDER_BASE" not in os.environ:
    print("EMBEDDER_BASE not found in .env file")
    os.environ["EMBEDDER_BASE"] = "sentence-transformers/multi-qa-mpnet-base-dot-v1"


# Logging level

import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

# 2. Importar librerías

import numpy as np
import pandas as pd
from pyhpo import Ontology
import matplotlib.pyplot as plt

In [2]:
# 3. Cargar conjuntos de datos

# Datasets

CORPUS_PATH = WS_ROUTE + "priv-RAG-DIIC/pubmed-hpo-pa-corpus/"
ABSTRACTS_PATH = CORPUS_PATH + "abstracts/"
PHENOTYPES_PATH = CORPUS_PATH + "phenotypes/"
EVAL_PATH = CORPUS_PATH + "evaluation/"

DF_ABSTRACTS = ABSTRACTS_PATH + "abstracts-12-05.csv"
DF_ABSTRACTS_TRAIN = ABSTRACTS_PATH + "abstracts-31-05-train.csv"

DF_EVAL_TRAIN = EVAL_PATH + "train_pairs-31-05.csv"
DF_EVAL_TEST = EVAL_PATH + "test_pairs-31-05.csv"

# Dataset de abstracts (crudo)

dfAbstracts = pd.read_csv(DF_ABSTRACTS, sep='\t', low_memory=False, na_values=[''])
dfAbstractsTrain = pd.read_csv(DF_ABSTRACTS_TRAIN, sep='\t', low_memory=False, na_values=[''])

dfPairsTrain = pd.read_csv(DF_EVAL_TRAIN, sep='\t', low_memory=False, na_values=[''])
dfPairsTest = pd.read_csv(DF_EVAL_TEST, sep='\t', low_memory=False, na_values=[''])

In [3]:
dfAbstracts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2541931 entries, 0 to 2541930
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   paperId        int64 
 1   phenotypeId    object
 2   phenotypeName  object
 3   title          object
 4   abstract       object
dtypes: int64(1), object(4)
memory usage: 97.0+ MB


In [4]:
# Tomar muestra pequeña estratificada para pruebas
dfProf = dfAbstracts.groupby('phenotypeId').apply(lambda x: x.sample(n=1, random_state=1))
dfProf = dfProf.sample(n=100, random_state=1)

# Quitar NA's

dfProf = dfProf.dropna(subset=['abstract'])


/tmp/ipykernel_1953452/2647373769.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  dfProf = dfAbstracts.groupby('phenotypeId').apply(lambda x: x.sample(n=1, random_state=1))


## Pipeline extractivo RAG

In [5]:
# 4. Estructura RAG

# 4.1. Librería Haystack

import haystack
from datasets import load_dataset
from haystack import Document
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.readers import ExtractiveReader
from haystack.components.embedders import SentenceTransformersDocumentEmbedder
from haystack.components.writers import DocumentWriter
from haystack.components.embedders import SentenceTransformersTextEmbedder

In [6]:
# 4.2 Embedder

embedderDocs = SentenceTransformersDocumentEmbedder(model=os.environ["PRITAMDEKA_BASE"])
embedderText = SentenceTransformersTextEmbedder(model=os.environ["PRITAMDEKA_BASE"])

# 4.3 Document Store

document_store = InMemoryDocumentStore(embedding_similarity_function='cosine')

# dataset: dfAbstracts
# columnas: paperId = PMID, phenotypeId : in HPO, title, abstract

# Cargar documentos en el Document Store

documents = [Document(content=row['abstract'], meta={'PMID': row['paperId'], 'phenotypeId': row['phenotypeId']}) for index, row in dfProf.iterrows()]

indexing_pipeline = Pipeline()

indexing_pipeline.add_component(instance=embedderDocs, name="embedder")
indexing_pipeline.add_component(instance=DocumentWriter(document_store=document_store), name="writer")
indexing_pipeline.connect("embedder.documents", "writer.documents")

indexing_pipeline.run({"documents": documents})

INFO - haystack.core.pipeline.base -  Warming up component embedder...
INFO - haystack.core.pipeline.pipeline -  Running component embedder


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

INFO - haystack.core.pipeline.pipeline -  Running component writer


{'writer': {'documents_written': 99}}

In [7]:
# 4.4 Retriever

retriever = InMemoryEmbeddingRetriever(document_store=document_store)

# 4.5 Prompt

from haystack.components.builders import PromptBuilder

template = """
Given the following information, provide a definition of the asked term and then describe its main characteristics.

Context:
{% for document in documents %}
    {{ document.content }}
{% endfor %}

Question: {{question}}
Answer:
"""

prompt_builder = PromptBuilder(template=template)

# 4.6 Reader

reader = ExtractiveReader()
reader.warm_up()

extractive_qa_pipeline = Pipeline()

extractive_qa_pipeline.add_component(instance=embedderText, name="embedder")
extractive_qa_pipeline.add_component(instance=retriever, name="retriever")
extractive_qa_pipeline.add_component(instance=reader, name="reader")

extractive_qa_pipeline.connect("embedder.embedding", "retriever.query_embedding")
extractive_qa_pipeline.connect("retriever.documents", "reader.documents")

🚅 Components
  - embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - reader: ExtractiveReader
🛤️ Connections
  - embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> reader.documents (List[Document])

In [8]:
query = dfAbstracts.sample(1)['phenotypeName'].values[0]
extractive_qa_pipeline.run(
    data={"embedder": {"text": query}, "retriever": {"top_k": 3}, "reader": {"query": query, "top_k": 2}}
)

INFO - haystack.core.pipeline.base -  Warming up component embedder...
INFO - haystack.core.pipeline.base -  Warming up component reader...
INFO - haystack.core.pipeline.pipeline -  Running component embedder


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO - haystack.core.pipeline.pipeline -  Running component retriever
INFO - haystack.core.pipeline.pipeline -  Running component reader


{'reader': {'answers': [ExtractedAnswer(query='Descending aortic dissection', score=0.503913722466887, data=None, document=None, context=None, document_offset=None, context_offset=None, meta={}),
   ExtractedAnswer(query='Descending aortic dissection', score=0.2991931140422821, data='70-84%', document=Document(id=1d0a9700d0f0bcd6807ca6ba71c77ead61b4047ac64bdb099adecae6ba936ade, content: 'BACKGROUND: Mandibuloacral Dysplasia with type B lipodystrophy (MADB) is a rare premature aging diso...', meta: {'PMID': 31856865, 'phenotypeId': 'HP:0006402'}, score: 0.3554295665105045), context=None, document_offset=ExtractedAnswer.Span(start=1075, end=1081), context_offset=None, meta={}),
   ExtractedAnswer(query='Descending aortic dissection', score=0.2809520959854126, data='Dyskeratosis congenita (DC) is an inherited multi-system disorder characterised by muco-cutaneous abnormalities, bone marrow failure and a predisposition to malignancy. Bone marrow failure is the principal cause of mortality a

## Evaluación

Evaluaremos el pipeline extractivo RAG comparando la similitud coseno de las queries aumentadas con la similitud Lin en HPO.
Los conjuntos de pares de evaluación train/test son:
  * train: [train_pairs-31-05.csv](bert-ft-data/evaluation/train_pairs-31-05.csv)
  * test:  [test_pairs-31-05.csv](bert-ft-data/evaluation/test_pairs-31-05.csv)

In [9]:
# 5. Conjuntos de evaluación -> dfPairs, dfTest

dfPairsTrain.info()
dfPairsTest.info()
dfPairsTest.head()

dfProfPairsTrain = dfPairsTrain.sample(100, random_state=1)
dfProfPairsTest = dfPairsTest.sample(100, random_state=1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   phenotype1  5000 non-null   object 
 1   phenotype2  5000 non-null   object 
 2   lin         5000 non-null   float64
 3   cos         5000 non-null   float64
dtypes: float64(2), object(2)
memory usage: 156.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   phenotype1  5000 non-null   object 
 1   phenotype2  5000 non-null   object 
 2   lin         5000 non-null   float64
dtypes: float64(1), object(2)
memory usage: 117.3+ KB


In [10]:
# 6. Añadir columnas para queries aumentadas
logging.getLogger("haystack").setLevel(logging.WARNING)

def augment_query(q):
    dout = extractive_qa_pipeline.run(data={"embedder": {"text": q}, "retriever": {"top_k": 3}, "reader": {"query": q, "top_k": 2, "no_answer":False}})
    #print(dout)
    xAns = dout["reader"]["answers"][0]
    qdef = xAns.data
    if qdef is None:
        return q
    aug_query = f"{q}: {qdef}"
    return aug_query

for df in [dfProfPairsTrain, dfProfPairsTest]:
    df['query1'] = df['phenotype1'].apply(augment_query)
    df['query2'] = df['phenotype2'].apply(augment_query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [11]:
dfProfPairsTest.head()

,phenotype1,phenotype2,lin,query1,query2
2764,Arterial thrombosis,Decreased resting energy expenditure,0.000043,Arterial thrombosis: Rarely persistent remnant...,Decreased resting energy expenditure: 0.017
4767,Short sternum,Difficulty climbing stairs,0.089940,Short sternum: Prenatal diagnosis in the subse...,Difficulty climbing stairs: Twelve percent (12...
3814,Sclerotic foci within carpal bones,Satyr ear,0.000080,Sclerotic foci within carpal bones: metaphysea...,Satyr ear: Common vestibulocochlear nerve was ...
3499,Wide nose,1-4 finger syndactyly,0.000052,Wide nose: clefts of nose are usually associat...,1-4 finger syndactyly: 2nd to 4th toes
2735,Osteoporosis of vertebrae,Arterial thrombosis,0.000043,Osteoporosis of vertebrae: The vertebral slip ...,Arterial thrombosis: Rarely persistent remnant...


In [12]:
# 7. Añadir columna similitud coseno original y aumentada

# 7.1 Cargar el modelo BERT

from sentence_transformers import SentenceTransformer, util

bertmodel = SentenceTransformer(os.environ["EMBEDDER_BASE"])

# 7.2 Calcular similitud coseno
for df_pairs in [dfProfPairsTrain, dfProfPairsTest]:
    df_pairs['cosOriginal'] = df_pairs.apply(lambda x: (util.cos_sim(bertmodel.encode(x['phenotype1']), bertmodel.encode(x['phenotype2'])).numpy()[0][0]), axis=1)
    df_pairs['cosAug'] = df_pairs.apply(lambda x: (util.cos_sim(bertmodel.encode(x['query1']), bertmodel.encode(x['query2'])).numpy()[0][0]), axis=1)

dfProfPairsTest.head()


,phenotype1,phenotype2,lin,query1,query2,cosOriginal,cosAug
2764,Arterial thrombosis,Decreased resting energy expenditure,0.000043,Arterial thrombosis: Rarely persistent remnant...,Decreased resting energy expenditure: 0.017,0.217571,0.052279
4767,Short sternum,Difficulty climbing stairs,0.089940,Short sternum: Prenatal diagnosis in the subse...,Difficulty climbing stairs: Twelve percent (12...,0.095960,0.169492
3814,Sclerotic foci within carpal bones,Satyr ear,0.000080,Sclerotic foci within carpal bones: metaphysea...,Satyr ear: Common vestibulocochlear nerve was ...,-0.026007,0.149100
3499,Wide nose,1-4 finger syndactyly,0.000052,Wide nose: clefts of nose are usually associat...,1-4 finger syndactyly: 2nd to 4th toes,0.149919,0.255675
2735,Osteoporosis of vertebrae,Arterial thrombosis,0.000043,Osteoporosis of vertebrae: The vertebral slip ...,Arterial thrombosis: Rarely persistent remnant...,0.309452,0.297882


In [13]:
# 8. Evaluar similitudes con MSE, Pearson y Spearman

from sklearn.metrics import mean_squared_error

dfTags = ['Train', 'Test']
dfList = [dfProfPairsTrain, dfProfPairsTest]
for i in range(2):
    df = dfList[i]
    for col in ['cosOriginal', 'cosAug']:
        print(f"Mean Squared Error ({dfTags[i]}, {col}): {mean_squared_error(df['lin'], df[col])}")
        print(f"Pearson Correlation ({dfTags[i]}, {col}): {df['lin'].corr(df[col], method='pearson')}")
        print(f"Spearman Correlation ({dfTags[i]}, {col}): {df['lin'].corr(df[col], method='spearman')}")

Mean Squared Error (Train, cosOriginal): 0.038382209407978776
Pearson Correlation (Train, cosOriginal): 0.09707854952595076
Spearman Correlation (Train, cosOriginal): 0.04107027871931328
Mean Squared Error (Train, cosAug): 0.050665107444810745
Pearson Correlation (Train, cosAug): 0.043095005035751234
Spearman Correlation (Train, cosAug): -0.12045560991821976
Mean Squared Error (Test, cosOriginal): 0.03164486426448545
Pearson Correlation (Test, cosOriginal): 0.1737957902982929
Spearman Correlation (Test, cosOriginal): 0.03621476203678203
Mean Squared Error (Test, cosAug): 0.045781259593037055
Pearson Correlation (Test, cosAug): 0.17060935836326724
Spearman Correlation (Test, cosAug): 0.2006443228691265


In [14]:
dfProfPairsTest.to_csv(EVAL_PATH + "test_pairs-31-05-augmented-2-prodia.csv", sep='\t', index=False)
dfProfPairsTrain.to_csv(EVAL_PATH + "train_pairs-31-05-augmented-2-prodia.csv", sep='\t', index=False)

# 5. Decisiones de diseño para el RAG

5. ¿Qué tipo de enfoque de RAG usar: extractivo, QA o generativo?
   * Caso extractivo: SentenceWindowRetriever. 
     * Ventaja: es más rápido.
     * Desventaja: solo utiliza sentencias de los documentos, no genera. Es incapaz de dar una definición si no la hay.
     * Tamaño en oraciones de los splits: en este caso 3.
     * Tamaño de la ventana: por defecto 3.
     * ¿Cómo generar la query aumentada a partir de la context window?